# 使用するカタログとスキーマを指定

In [0]:
USE CATALOG handson;
USE SCHEMA sports_orders;

# シルバーテーブル初期作成（スキーマは同じ）

In [0]:
CREATE TABLE IF NOT EXISTS silver_sports_orders (
  order_id INT,
  customer_name STRING,
  region STRING,
  order_date DATE,
  product STRING,
  category STRING,
  quantity INT,
  price DOUBLE
);

# 更新後のBronzeテーブルからSilverテーブルを作成

`MERGE INTO`：ターゲットのDeltaテーブルにソーステーブルからのデータをマージするための汎用SQL文 [(参考資料)](https://qiita.com/taka_yayoi/items/4376b3403a3dbfdfe89f)

In [0]:
MERGE INTO silver_sports_orders AS target
USING (
  SELECT
    order_id,

    -- 顧客名の前後の空白を削除
    TRIM(customer_name) AS customer_name,

    -- 地域名の前後の空白を削除
    TRIM(region) AS region,
    
    -- 注文日が想定範囲外の値はNULLとする
    CASE
      WHEN TO_DATE(order_date) BETWEEN DATE('2020-01-01') AND CURRENT_DATE() THEN TO_DATE(order_date)
      ELSE NULL
    END AS order_date,
    
    -- 商品名の前後の空白を削除
    TRIM(product) AS product,
    
    -- カテゴリ名の前後の空白を削除
    TRIM(category) AS category,

    -- 数量が負の場合はNULLとする
    CASE WHEN quantity >= 0 THEN quantity ELSE NULL END AS quantity,

    -- 価格が負の場合はNULLとする
    CASE WHEN price >= 0 THEN price ELSE NULL END AS price
  FROM bronze_sports_orders
  WHERE order_id IS NOT NULL  -- 注文IDがNULLのレコードは除外
) AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN
  UPDATE SET
    target.customer_name = source.customer_name,
    target.region = source.region,
    target.order_date = source.order_date,
    target.product = source.product,
    target.category = source.category,
    target.quantity = source.quantity,
    target.price = source.price
WHEN NOT MATCHED THEN
  INSERT *
WHEN NOT MATCHED BY SOURCE THEN
  DELETE
;